In [486]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [588]:
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [545]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 25 columns):
id            70000 non-null int64
date          70000 non-null object
street_id     70000 non-null int64
build_tech    49121 non-null float64
floor         70000 non-null int64
area          70000 non-null int64
rooms         70000 non-null int64
balcon        70000 non-null int64
metro_dist    66550 non-null float64
g_lift        48924 non-null float64
n_photos      70000 non-null int64
kw1           70000 non-null int64
kw2           70000 non-null int64
kw3           70000 non-null int64
kw4           70000 non-null int64
kw5           70000 non-null int64
kw6           70000 non-null int64
kw7           70000 non-null int64
kw8           70000 non-null int64
kw9           70000 non-null int64
kw10          70000 non-null int64
kw11          70000 non-null int64
kw12          70000 non-null int64
kw13          70000 non-null int64
price         70000 non-null int64
dtypes: f

In [546]:
np.unique(train_df['g_lift'])

array([ 0.,  1., nan, ..., nan, nan, nan])

In [547]:
category_features = ['street_id','balcon','build_tech']
num_features = ['metro_dist', 'rooms','floor', 'area', 'n_photos']
bool_features = ['g_lift']
y = ['price']
x = train_df.iloc[:,:-1]
for i in range(14):
    bool_features.append('kw' + str(i))

# Fill nans

In [548]:
train_df = train_df.drop(['date','id'], axis = 1)
values = {'metro_dist': train_df['metro_dist'].mean()}
train_df.fillna(value=values, inplace=True)

In [549]:
# #восстановление пропусков
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

cols = ['g_lift','build_tech']
pred_columns = []
for col in cols:
     x = train_df[train_df[col].notna()]
     x_test = train_df[train_df[col].isna()]
     y = x[col]
     print(x_test.shape)
     x = x.drop(['g_lift', 'build_tech'], axis = 1)
     x_test = x_test.drop(['g_lift', 'build_tech'], axis = 1)

     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

     neigh = KNeighborsClassifier()
     gc = GridSearchCV(estimator=neigh,
                 param_grid={'n_neighbors': [3,5,7,10,13,15,20]})
     gc.fit(X_train, y_train)
     gc.best_estimator_.get_params
     print(gc.best_estimator_.score(X_test,y_test))
     predictions = pd.Series(gc.best_estimator_.predict(x_test))
     print(predictions.shape)
     pred_columns.append(predictions) 
     


(21076, 23)
0.4940724388848009
(21076,)
(20879, 23)
0.6256005211302011
(20879,)


In [550]:
g_lift_col = train_df['g_lift'].to_list()
build_tech_col = train_df['build_tech'].to_list()

In [551]:
k = 0
for i in range(len(g_lift_col)):
    if pd.isna(g_lift_col[i]):
        g_lift_col[i] = pred_columns[0][k]
        k+=1
k = 0
for i in range(len(build_tech_col)):
    if pd.isna(build_tech_col[i]):
        build_tech_col[i] = pred_columns[1][k]
        k+=1

In [552]:
train_df.fillna(value = {'g_lift': pd.Series(g_lift_col), 'build_tech': pd.Series(build_tech_col)}, inplace=True)

In [553]:
train_df = train_df.astype({'build_tech': 'object', 'g_lift' : 'int64', 'balcon':'object'}, copy=False)

# Features processing

In [554]:
cat_indices = [1, 5]
one_hot = pd.get_dummies(train_df[train_df.columns[cat_indices]])
train_df = train_df.drop(train_df.columns[cat_indices],axis = 1)
train_df = train_df.join(one_hot)

In [555]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 27 columns):
street_id         70000 non-null int64
floor             70000 non-null int64
area              70000 non-null int64
rooms             70000 non-null int64
metro_dist        70000 non-null float64
g_lift            70000 non-null int64
n_photos          70000 non-null int64
kw1               70000 non-null int64
kw2               70000 non-null int64
kw3               70000 non-null int64
kw4               70000 non-null int64
kw5               70000 non-null int64
kw6               70000 non-null int64
kw7               70000 non-null int64
kw8               70000 non-null int64
kw9               70000 non-null int64
kw10              70000 non-null int64
kw11              70000 non-null int64
kw12              70000 non-null int64
kw13              70000 non-null int64
price             70000 non-null int64
build_tech_0.0    70000 non-null uint8
build_tech_1.0    70000 non-nul

In [556]:
num_inds = [1,2,3,4, 6]

In [557]:
"""hash_space = len(np.unique(train_df['street_id']))*3
street_id_arr = np.array(train_df['street_id'])
for i in range(len(street_id_arr)):
    street_id_arr[i] =  hash(street_id_arr[i]) % hash_space
train_df = train_df.drop(train_df.columns[0], axis = 1)
train_df.loc[:,"street_id"] = pd.Series(street_id_arr, index = train_df.index)"""

'hash_space = len(np.unique(train_df[\'street_id\']))*3\nstreet_id_arr = np.array(train_df[\'street_id\'])\nfor i in range(len(street_id_arr)):\n    street_id_arr[i] =  hash(street_id_arr[i]) % hash_space\ntrain_df = train_df.drop(train_df.columns[0], axis = 1)\ntrain_df.loc[:,"street_id"] = pd.Series(street_id_arr, index = train_df.index)'

In [577]:
scaler = StandardScaler()#нормализация данных
num_features = pd.DataFrame(scaler.fit_transform(train_df[train_df.columns[num_inds]]), columns = ['floor', 'area', 'rooms', 'metro_dist','n_photos'])

In [578]:
train_df = train_df.drop(train_df.columns[num_inds],axis = 1)
train_df = train_df.join(num_features)

# Train

In [579]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 27 columns):
street_id         70000 non-null int64
g_lift            70000 non-null int64
kw1               70000 non-null int64
kw2               70000 non-null int64
kw3               70000 non-null int64
kw4               70000 non-null int64
kw5               70000 non-null int64
kw6               70000 non-null int64
kw7               70000 non-null int64
kw8               70000 non-null int64
kw9               70000 non-null int64
kw10              70000 non-null int64
kw11              70000 non-null int64
kw12              70000 non-null int64
kw13              70000 non-null int64
price             70000 non-null int64
build_tech_0.0    70000 non-null uint8
build_tech_1.0    70000 non-null uint8
build_tech_2.0    70000 non-null uint8
balcon_0          70000 non-null uint8
balcon_1          70000 non-null uint8
balcon_2          70000 non-null uint8
floor             70000 non-null 

In [607]:
Y = train_df['price']
X = train_df.drop('price', axis = 1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
from lightgbm import LGBMRegressor
LGBM = LGBMRegressor(n_estimators=500, boosting_type ='goss')
LGBM.fit(X_train, y_train)

LGBMRegressor(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [608]:
metric = np.sqrt(mean_squared_error(boosting_model.predict(X_test),y_test))
metric

2372649.4113129834

In [583]:
from sklearn.metrics import mean_squared_error
metric = np.sqrt(mean_squared_error(LGBM.predict(X_test),y_test))
metric

2343785.9917051047

# Submission

In [589]:
test_df = test_df.drop(['date','id'], axis = 1)
values = {'metro_dist': test_df['metro_dist'].mean()}
test_df.fillna(value=values, inplace=True)

In [590]:
# #восстановление пропусков
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

cols = ['g_lift','build_tech']
pred_columns = []
for col in cols:
     x = test_df[test_df[col].notna()]
     x_test = test_df[test_df[col].isna()]
     y = x[col]
     print(x_test.shape)
     x = x.drop(['g_lift', 'build_tech'], axis = 1)
     x_test = x_test.drop(['g_lift', 'build_tech'], axis = 1)

     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

     neigh = KNeighborsClassifier()
     gc = GridSearchCV(estimator=neigh,
                 param_grid={'n_neighbors': [3,5,7,10,13,15,20]})
     gc.fit(X_train, y_train)
     gc.best_estimator_.get_params
     print(gc.best_estimator_.score(X_test,y_test))
     predictions = pd.Series(gc.best_estimator_.predict(x_test))
     print(predictions.shape)
     pred_columns.append(predictions) 
     


(8916, 22)
0.49800796812749004
(8916,)
(8856, 22)
0.684449489216799
(8856,)


In [591]:
g_lift_col = test_df['g_lift'].to_list()
build_tech_col = test_df['build_tech'].to_list()

In [592]:
k = 0
for i in range(len(g_lift_col)):
    if pd.isna(g_lift_col[i]):
        g_lift_col[i] = pred_columns[0][k]
        k+=1
k = 0
for i in range(len(build_tech_col)):
    if pd.isna(build_tech_col[i]):
        build_tech_col[i] = pred_columns[1][k]
        k+=1

In [593]:
test_df.fillna(value = {'g_lift': pd.Series(g_lift_col), 'build_tech': pd.Series(build_tech_col)}, inplace=True)

In [594]:
test_df = test_df.astype({'build_tech': 'object', 'g_lift' : 'int64', 'balcon':'object'}, copy=False)

In [595]:
cat_indices = [1, 5]
one_hot = pd.get_dummies(test_df[test_df.columns[cat_indices]])
test_df = test_df.drop(test_df.columns[cat_indices],axis = 1)
test_df = test_df.join(one_hot)

In [596]:
"""hash_space = len(np.unique(test_df['street_id']))*3
street_id_arr = np.array(test_df['street_id'])
for i in range(len(street_id_arr)):
    street_id_arr[i] =  hash(street_id_arr[i]) % hash_space
test_df = test_df.drop(test_df.columns[0], axis = 1)
test_df.loc[:,"street_id"] = pd.Series(street_id_arr, index = test_df.index)"""

'hash_space = len(np.unique(test_df[\'street_id\']))*3\nstreet_id_arr = np.array(test_df[\'street_id\'])\nfor i in range(len(street_id_arr)):\n    street_id_arr[i] =  hash(street_id_arr[i]) % hash_space\ntest_df = test_df.drop(test_df.columns[0], axis = 1)\ntest_df.loc[:,"street_id"] = pd.Series(street_id_arr, index = test_df.index)'

In [597]:
scaler = StandardScaler()#нормализация данных
num_features = pd.DataFrame(scaler.fit_transform(test_df[test_df.columns[num_inds]]), columns = ['floor', 'area', 'rooms', 'metro_dist','n_photos'])
test_df = test_df.drop(test_df.columns[num_inds],axis = 1)
test_df = test_df.join(num_features)

In [601]:
submit = pd.read_csv('SampleSubmission.csv')
submit['price']=LGBM.predict(test_df)

In [602]:
submit.to_csv('./my_submission9.csv', index = False)

In [600]:
train_df

,street_id,g_lift,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,...,build_tech_1.0,build_tech_2.0,balcon_0,balcon_1,balcon_2,floor,area,rooms,metro_dist,n_photos
0,560,1,0,1,0,0,0,0,0,0,...,1,0,1,0,0,-0.838549,0.380205,1.063414,0.971499,1.302402
1,232,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,-0.582362,0.436463,1.063414,0.351639,0.272678
2,453,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,-0.069987,-0.013602,-0.142056,0.971499,-0.757045
3,44,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,-1.094736,-1.082507,-1.347527,0.971499,-1.271907
4,74,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,2.748075,-0.463668,-1.347527,-0.268221,-0.757045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,598,1,0,0,0,1,0,0,0,0,...,1,0,1,0,0,-1.094736,0.492721,1.063414,-2.127801,1.817263
69996,401,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,-0.069987,-1.251282,-1.347527,0.971499,-0.757045
69997,59,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,-0.326174,-1.026249,-1.347527,0.971499,-0.757045
69998,253,1,0,1,0,0,0,0,0,0,...,1,0,1,0,0,-0.582362,-0.013602,-0.142056,-2.127801,0.787540
